# Data Aggregation and Group Operations

*** 

Author: Jiacheng

Date: 2019-05-19

***

In [1]:
import numpy as np
import pandas as pd

---

## 1. GroupBy Mechanics
## GroupBy 机制

* ### 1. 入门

In [14]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.216621,-0.153862
1,a,two,-0.518636,-2.056840
2,b,one,0.576604,-0.624191
3,b,two,-1.819788,1.260355
4,a,one,-0.484164,0.167761


In [4]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [5]:
grouped.mean()

key1
a   -1.186781
b   -1.647323
Name: data1, dtype: float64

In [6]:
# 多重分组
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.427170
      two    -2.706003
b     one    -1.429791
      two    -1.864855
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,one,two
key1,,
a,-0.427170,-2.706003
b,-1.429791,-1.864855


In [8]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean().unstack()

,2005,2006
California,-2.706003,-1.429791
Ohio,-1.616079,0.512964


In [9]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.427170 -0.036146
     two  -2.706003  0.310918
b    one  -1.429791 -0.623735
     two  -1.864855  0.546224

In [10]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

* ### 2. 对分组进行迭代

In [11]:
# 迭代产生一组二元元组
for name, group in df.groupby('key1'):
     print(name)
     print(group)

a
  key1 key2     data1     data2
0    a  one -1.367304  0.180706
1    a  two -2.706003  0.310918
4    a  one  0.512964 -0.252997
b
  key1 key2     data1     data2
2    b  one -1.429791 -0.623735
3    b  two -1.864855  0.546224


In [12]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -1.367304  0.180706
4    a  one  0.512964 -0.252997
('a', 'two')
  key1 key2     data1     data2
1    a  two -2.706003  0.310918
('b', 'one')
  key1 key2     data1     data2
2    b  one -1.429791 -0.623735
('b', 'two')
  key1 key2     data1     data2
3    b  two -1.864855  0.546224


In [13]:
pieces = dict(list(df.groupby('key1')))

In [14]:
pieces

{'a':   key1 key2     data1     data2
 0    a  one -1.367304  0.180706
 1    a  two -2.706003  0.310918
 4    a  one  0.512964 -0.252997, 'b':   key1 key2     data1     data2
 2    b  one -1.429791 -0.623735
 3    b  two -1.864855  0.546224}

In [15]:
pieces['a']

,key1,key2,data1,data2
0,a,one,-1.367304,0.180706
1,a,two,-2.706003,0.310918
4,a,one,0.512964,-0.252997


In [16]:
# 将根据dtypes 进行分组
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [17]:
grouped = df.groupby(df.dtypes, axis=1)

In [18]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -1.367304  0.180706
1 -2.706003  0.310918
2 -1.429791 -0.623735
3 -1.864855  0.546224
4  0.512964 -0.252997
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


* ### 3. 选取一列或列的子集

In [19]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.036146
     two   0.310918
b    one  -0.623735
     two   0.546224

In [20]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [21]:
s_grouped.mean()

key1  key2
a     one    -0.036146
      two     0.310918
b     one    -0.623735
      two     0.546224
Name: data2, dtype: float64

* ### 4. 通过字典或Series进行分组

In [2]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1,2]] = np.nan
people


,a,b,c,d,e
Joe,0.710084,0.684721,0.699034,2.293984,0.980396
Steve,-0.238650,-1.783223,0.095479,1.334715,1.251450
Wes,0.275352,NaN,NaN,-2.061595,0.228844
Jim,-0.275153,-1.188928,0.727625,0.849615,-2.722091
Travis,-1.372959,-0.963568,-1.010431,-0.302377,0.313110


In [3]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,2.993018,2.375200
Steve,1.430194,-0.770423
Wes,-2.061595,0.504196
Jim,1.577240,-4.186171
Travis,-1.312808,-2.023417


In [4]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [5]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


* ### 5. 通过函数进行分组

In [8]:
# 根据名字长度分类
people.groupby(len).sum()

,a,b,c,d,e
3,0.710283,-0.504207,1.426659,1.082004,-1.512851
5,-0.238650,-1.783223,0.095479,1.334715,1.251450
6,-1.372959,-0.963568,-1.010431,-0.302377,0.313110


In [9]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.275352  0.684721  0.699034 -2.061595  0.228844
  two -0.275153 -1.188928  0.727625  0.849615 -2.722091
5 one -0.238650 -1.783223  0.095479  1.334715  1.251450
6 two -1.372959 -0.963568 -1.010431 -0.302377  0.313110

* ### 6. 根据索引级别分组

In [10]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.020641 -0.978781  0.629434  0.166689 -0.889020
1      0.909967 -0.837581 -0.640614  1.647019  0.536708
2     -1.207694 -0.352867 -1.096377 -0.131898  0.390136
3     -0.058926  0.452547 -1.092684 -0.542241  0.856290

In [12]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


---

## 2. Data Aggregation
## 数据聚合

* ### 1. 入门

In [15]:
df

,key1,key2,data1,data2
0,a,one,0.216621,-0.153862
1,a,two,-0.518636,-2.056840
2,b,one,0.576604,-0.624191
3,b,two,-1.819788,1.260355
4,a,one,-0.484164,0.167761


In [16]:
grouped = df.groupby('key1')

In [18]:
# 分位数
grouped['data1'].quantile(0.9)

key1
a    0.076464
b    0.336965
Name: data1, dtype: float64

In [19]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [20]:
# 使用自己的函数
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.735257,2.224601
b,2.396392,1.884547


In [21]:
grouped.describe()

data1                                                             \
     count      mean       std       min      25%       50%       75%   
key1                                                                    
a      3.0 -0.262060  0.414908 -0.518636 -0.50140 -0.484164 -0.133772   
b      2.0 -0.621592  1.694505 -1.819788 -1.22069 -0.621592 -0.022494   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.216621   3.0 -0.680980  1.202332 -2.056840 -1.105351 -0.153862   
b     0.576604   2.0  0.318082  1.332576 -0.624191 -0.153055  0.318082   

                          
           75%       max  
key1                      
a     0.006950  0.167761  
b     0.789219  1.260355

* ### 2. 面向列的多函数应用

In [22]:
tips = pd.read_csv('../examples/tips.csv')

In [23]:
tips[:10]

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
5,25.29,4.71,No,Sun,Dinner,4
6,8.77,2.00,No,Sun,Dinner,2
7,26.88,3.12,No,Sun,Dinner,4
8,15.04,1.96,No,Sun,Dinner,2
9,14.78,3.23,No,Sun,Dinner,2


In [24]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:10]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240
6,8.77,2.00,No,Sun,Dinner,2,0.228050
7,26.88,3.12,No,Sun,Dinner,4,0.116071
8,15.04,1.96,No,Sun,Dinner,2,0.130319
9,14.78,3.23,No,Sun,Dinner,2,0.218539


In [25]:
grouped = tips.groupby(['day', 'smoker'])

In [26]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [27]:
# 传入一组函数
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [28]:
# 元组第一个元素作为列名
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [29]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [30]:
# 对不同的列应用不同的函数
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [31]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

* ### 3. 以“没有行索引”的形式返回聚合数据

In [32]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


---

## 3. Apply: General split-apply-combine
## apply：一般性的“拆分－应用－合并”

* ### 1. 入门

In [33]:
# 选出最高的n个tip_pct值
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [34]:
top(tips, n=7)

,total_bill,tip,smoker,day,time,size,tip_pct
149,7.51,2.00,No,Thur,Lunch,2,0.266312
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [35]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [39]:
tips.groupby(['smoker', 'day']).apply(top, n=2, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  91        22.49   3.50     No   Fri  Dinner     2  0.155625
            94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  59        48.27   6.73     No   Sat  Dinner     4  0.139424
            212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  112       38.07   4.00     No   Sun  Dinner     3  0.105070
            156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 85        34.83   5.17     No  Thur   Lunch     4  0.148435
            142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
            95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
            170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
            182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 83        32.68   5.00    Yes  Thur   Lunch     2  0.152999
            197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [40]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [41]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

* ### 2. 禁止分组键

In [42]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


* ### 3. 分位数和桶分析